In [1]:
# NI-DAQmx Python Documentation: https://nidaqmx-python.readthedocs.io/en/latest/index.html
# NI USB-621x User Manual: https://www.ni.com/docs/en-US/bundle/usb-621x-manual/resource/usb-621x-manual.pdf
import matplotlib.pyplot as plt
import numpy as np
import nidaqmx
import math
import time
import pandas as pd
import os
import datetime

In [2]:
"""
import ctypes
try:
    nidaq = ctypes.windll.nicaiu
except:
    raise Exception("No se pudo abrir la libreria DAQmx")
"""

'\nimport ctypes\ntry:\n    nidaq = ctypes.windll.nicaiu\nexcept:\n    raise Exception("No se pudo abrir la libreria DAQmx")\n'

In [3]:
#para saber el ID de la placa conectada (DevX)
system = nidaqmx.system.System.local()
for device in system.devices:
    print(device)



DaqNotFoundError: Could not find an installation of NI-DAQmx. Please ensure that NI-DAQmx is installed on this machine or contact National Instruments for support.

In [ ]:
#para setear (y preguntar) el modo y rango de un canal analógico
modoRSE= nidaqmx.constants.TerminalConfiguration.RSE # puede ser DIFF o DIFFERENTIAL 
modoDIFF= nidaqmx.constants.TerminalConfiguration.DIFF # puede ser DIFF o DIFFERENTIAL

with nidaqmx.Task() as task:
    ai_channel = task.ai_channels.add_ai_voltage_chan("Dev2/ai1",max_val=10,min_val=-10,terminal_config= modoRSE )
    print(ai_channel.ai_term_cfg)    # puede ser DIFF o DIFFERENTIAL
    print(ai_channel.ai_max)
    print(ai_channel.ai_min)	

In [ ]:
with nidaqmx.Task() as task:
    ai_channel = task.ai_channels.add_ai_voltage_chan("Dev2/ai1",max_val=10,min_val=-10)
    print(ai_channel.ai_term_cfg)
    print(ai_channel.ai_max)
    print(ai_channel.ai_min)

In [ ]:

## Medicion continua
def medicion_continua(duracion, fs):
    cant_puntos = int(duracion*fs)
    with nidaqmx.Task() as task:
        modoRSE= nidaqmx.constants.TerminalConfiguration.RSE # puede ser DIFF o DIFFERENTIAL 
        modoDIFF= nidaqmx.constants.TerminalConfiguration.DIFF # puede ser DIFF o DIFFERENTIAL 
        task.ai_channels.add_ai_voltage_chan("Dev2/ai1", terminal_config =  modoDIFF)
        task.ai_channels.add_ai_voltage_chan("Dev2/ai2", terminal_config =  modoDIFF)
        task.ai_channels.add_ai_voltage_chan("Dev2/ai4", terminal_config =  modoDIFF)
        task.ai_channels.add_ai_voltage_chan("Dev2/ai5", terminal_config =  modoRSE)
        task.timing.cfg_samp_clk_timing(fs, sample_mode = nidaqmx.constants.AcquisitionType.CONTINUOUS)
        task.start()
        t0 = time.time()
        total = 0
        data =[[],[],[],[],[]]
        Time = []
        while total<cant_puntos:
            time.sleep(0.08)
            datos = task.read(number_of_samples_per_channel=nidaqmx.constants.READ_ALL_AVAILABLE)           
            data[0].append(datos[0])
            data[1].append(datos[1])
            data[2].append(datos[2])
            data[3].append(datos[3])
            total = total + len(datos[0])
            t1 = time.time()
        data[4] = np.linspace(0,t1-t0,len(data[0]))
        return data
    
def medicion_una_vez(duracion, fs):
    cant_puntos = int(duracion*fs)
    with nidaqmx.Task() as task:
        modoRSE= nidaqmx.constants.TerminalConfiguration.RSE # puede ser DIFF o DIFFERENTIAL 
        modoDIFF= nidaqmx.constants.TerminalConfiguration.DIFF # puede ser DIFF o DIFFERENTIAL 
        task.ai_channels.add_ai_voltage_chan("Dev2/ai1", terminal_config =  modoDIFF)
        task.ai_channels.add_ai_voltage_chan("Dev2/ai2", terminal_config =  modoDIFF)
        task.ai_channels.add_ai_voltage_chan("Dev2/ai4", terminal_config =  modoDIFF)
        task.ai_channels.add_ai_voltage_chan("Dev2/ai5", terminal_config =  modoRSE)
               
        task.timing.cfg_samp_clk_timing(fs,samps_per_chan = cant_puntos,
                                        sample_mode = nidaqmx.constants.AcquisitionType.FINITE)
        
        datos = task.read(number_of_samples_per_channel=nidaqmx.constants.READ_ALL_AVAILABLE, timeout=duracion+0.1)           
    datos = np.asarray(datos)    
    return datos

fs = 40000 #Frecuencia de muestreo
duracion = 300 #segundos
data = medicion_continua(duracion, fs)


# plt.plot(data[2], data[3])

df = pd.DataFrame(data)
df = df.T
df.columns = ["B","H","V_platino","V_R","Time"]


df.to_csv(f'datos_array{datetime.datetime.fromtimestamp(time.time()).strftime("%d_%H_%M_%S")}.csv', index=False)
df

In [ ]:
for i in range(200,3600,50):
                   plt.plot(df["H"][i],df["B"][i])
                   plt.show()


In [ ]:
plt.scatter(df["V_platino"][300],df["V_R"][300])
plt.grid()

In [ ]:
a = 1
b = -100
def temperatura(res):
    return res * a + b

def temp_kelv(res):
    return res * 0.00385 + 273.23 - 0.385

def resistencia_platino(V_plat, V_R, res=2000):
    return np.abs(V_plat / V_R) * res

In [ ]:
v_arr = [[],[]]

for i in range(len(df["V_platino"]) - 1):
    v_arr[0].append((np.std(df["V_platino"][i+1])))
    v_arr[1].append((np.std(df["V_R"][i+1])))




In [ ]:
def PT100_res2temp_interp(R): #np.mean
    data = np.loadtxt('Pt100_resistencia_temperatura.csv',delimiter=',') 
    temperature_vals = data[:,0] # en Celsius
    resistance_vals = data[:,1] #en Ohm
    return np.interp(R, resistance_vals, temperature_vals)

R = resistencia_platino(np.array(v_arr[0]),  np.array(v_arr[1]))
plt.scatter(R, PT100_res2temp_interp(R))
plt.ylabel('Temperatura [C]')
plt.xlabel('Resistencia Pt100 [Ohm]')
plt.grid(True)


In [ ]:
temperatura(100)

In [ ]:
temperatura(205)

In [ ]:


# def daq_conteo(duracion):
#     with nidaqmx.Task() as task:

#         # Configuro la task para edge counting
#         task.ci_channels.add_ci_count_edges_chan(counter=ci_chan1,
#             name_to_assign_to_channel="",
#             edge=nidaqmx.constants.Edge.RISING,
#             initial_count=0)
        
#         # arranco la task
#         task.start()
#         counts = [0]
#         t0 = time.time()
#         try:
#             while time.time() - t0 < duracion:
#                 count = task.ci_channels[0].ci_count
#                 print(f"{time.time()-t0:.2f}s {count-counts[-1]} {count}")
#                 counts.append(count)
#                 time.sleep(0.2)
                
#         except KeyboardInterrupt:
#             pass
        
#         finally:
#             task.stop()
            
#     return counts  

# duracion = 10 # segundos
# y = daq_conteo(duracion)
# plt.plot(y)
# plt.grid()
# plt.show()

# ## Medición con trigger
# # Pinout: 
# # NiUSB6210 
# # PFI0: 1
# # D GND: 5 o 11
# from nidaqmx.constants import AcquisitionType, Edge

# def medicion__una_vez_con_trigger(duracion, fs):
#     cant_puntos = int(duracion*fs)
#     with nidaqmx.Task() as task:
#         # Canal analógico a medir
#         modo= nidaqmx.constants.TerminalConfiguration.DIFFERENTIAL # puede ser DIFF o DIFFERENTIAL PREGUNTAR
#         task.ai_channels.add_ai_voltage_chan("Dev1/ai1", terminal_config = modo)
   
#         # Configuración de muestreo
#         task.timing.cfg_samp_clk_timing(rate=fs, sample_mode=AcquisitionType.FINITE, samps_per_chan=cant_puntos)
   
#         # Configurar trigger digital
#         task.triggers.start_trigger.cfg_dig_edge_start_trig(trigger_source="/Dev1/PFI0", trigger_edge=Edge.RISING)
   
#         task.start()
#         datos = task.read(number_of_samples_per_channel=cant_puntos)
#     datos = np.asarray(datos)
#     return datos

# duracion = 1 #segundos
# fs = 250000 #Frecuencia de muestreo
# y = medicion__una_vez_con_trigger(duracion, fs)
# plt.plot(y)
# plt.grid()
# plt.show()